In [1]:
# --- 1.1. INSTALACIÓN DE BIBLIOTECAS ---
# Instala las bibliotecas principales que usaremos
!pip install -q 'segment-anything'  # Biblioteca oficial de SAM
!pip install -q opencv-python-headless
!pip install -q shapely             # Para la fusión de polígonos
!pip install -q matplotlib
!pip install -q ipywidgets          # Para los sliders interactivos

# --- 1.2. IMPORTACIÓN DE BIBLIOTECAS ---
import numpy as np
import cv2
import os
import requests
import torch
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import ipywidgets as widgets
from ipywidgets import interactive, HBox, VBox
from IPython.display import display, clear_output
from shapely.geometry import Polygon
from shapely.ops import unary_union

print("Bibliotecas instaladas e importadas.")

# --- 1.3. MONTAR GOOGLE DRIVE (Ejecutar solo en Google Colab) ---
try:
    from google.colab import drive
    drive.mount('/content/drive')
    # La ruta de Google Drive se traduce así en Colab
    image_path = "/content/drive/My Drive/0.- Archivos/Software/Proyecto digitalizacion geologica/1.- Imagenes/santarosa.png"
    print("Google Drive montado.")
except ImportError:
    print("Google Colab no detectado. Usando ruta local.")
    # Si ejecutas esto localmente, asegúrate de que G: sea accesible
    image_path = r"G:\Mi unidad\0.- Archivos\Software\Proyecto digitalizacion geologica\1.- Imagenes\santarosa.png"

# --- 1.4. CARGAR LA IMAGEN ---
try:
    # Cargar con PIL (para ajustes interactivos)
    img_pil = Image.open(image_path).convert("RGB")

    # Cargar con OpenCV (para SAM y visualización)
    # SAM espera una imagen RGB en formato numpy
    img_rgb = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)

    print(f"Imagen cargada exitosamente: {image_path}")
    print(f"Dimensiones: {img_pil.size}")
except Exception as e:
    print(f"Error: No se pudo cargar la imagen en la ruta: {image_path}")
    print(e)

# Variable global para guardar la imagen ajustada
adjusted_image_pil = img_pil.copy()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.5 MB/s eta 0:00:00
Bibliotecas instaladas e importadas.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado.
Imagen cargada exitosamente: /content/drive/My Drive/0.- Archivos/Software/Proyecto digitalizacion geologica/1.- Imagenes/santarosa.png
Dimensiones: (3032, 3025)


In [17]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

print("Bloque 2: Aplicando Pre-procesamiento Avanzado...")

# Tomamos la imagen original cargada en el Bloque 1 (img_rgb)
img_to_process = img_rgb.copy()

# --- 1. Filtro Bilateral ---
# d=9: Diámetro del vecindario de píxeles.
# sigmaColor=75: Qué tanto se mezclan los colores.
# sigmaSpace=75: Qué tan lejos influyen los píxeles.
# Esto suavizará los colores DENTRO de los polígonos, pero PRESERVARÁ los bordes.
print("Aplicando Filtro Bilateral (Paso 1/3)...")
img_bilateral = cv2.bilateralFilter(img_to_process, d=9, sigmaColor=75, sigmaSpace=75)

# --- 2. Operación Morfológica de "Apertura" ---
# Elimina ruido pequeño (como líneas de texto o bordes muy finos)
# Kernel de 3x3.
print("Aplicando Apertura Morfológica (Paso 2/3)...")
kernel = np.ones((3, 3), np.uint8)
img_open = cv2.morphologyEx(img_bilateral, cv2.MORPH_OPEN, kernel, iterations=1)

# --- 3. Operación Morfológica de "Cierre" ---
# Rellena pequeños agujeros DENTRO de los polígonos
# Kernel de 3x3.
print("Aplicando Cierre Morfológico (Paso 3/3)...")
kernel = np.ones((3, 3), np.uint8)
img_closed = cv2.morphologyEx(img_open, cv2.MORPH_CLOSE, kernel, iterations=1)

print("Pre-procesamiento completado.")

# --- Visualización de la Comparación ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))

ax1.imshow(img_rgb)
ax1.set_title("Imagen Original")
ax1.axis('off')

ax2.imshow(img_closed)
ax2.set_title("Imagen Pre-procesada (Limpiada para SAM)")
ax2.axis('off')

plt.tight_layout()
plt.show()

# --- Actualizar la variable global ---
# Convertimos la imagen procesada (numpy) de vuelta a PIL
# para que los siguientes bloques (3, 4, 5) la usen.
adjusted_image_pil = Image.fromarray(img_closed)

Output hidden; open in https://colab.research.google.com to view.

In [18]:
# --- 3.1. Descargar Checkpoint del Modelo ---
checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth"
checkpoint_path = "sam_vit_h.pth"

# Descargar solo si no existe
if not os.path.exists(checkpoint_path):
    print("Descargando checkpoint de SAM (ViT-Huge)...")
    try:
        response = requests.get(checkpoint_url)
        response.raise_for_status() # Asegurarse de que la descarga fue exitosa
        with open(checkpoint_path, 'wb') as f:
            f.write(response.content)
        print("Descarga completa.")
    except Exception as e:
        print(f"Error al descargar el modelo: {e}")

# --- 3.2. Configurar Modelo y Dispositivo ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_type = "vit_h"

if os.path.exists(checkpoint_path):
    print(f"Cargando modelo SAM en el dispositivo: {device}")
    sam = sam_model_registry[model_type](checkpoint=checkpoint_path)
    sam.to(device=device)
    print("Modelo cargado exitosamente.")

    # --- 3.3. Configurar el Generador Automático de Máscaras ---
    # ¡PARÁMETROS REVISADOS! Los bajamos para ser menos estrictos.
    print("Configurando generador con parámetros más relajados...")
    mask_generator = SamAutomaticMaskGenerator(
        model=sam,
        points_per_side=32,
        pred_iou_thresh=0.80,        # Bajado de 0.86 (más inclusivo)
        stability_score_thresh=0.85, # Bajado de 0.92 (mucho más inclusivo)
        crop_n_layers=1,
        crop_n_points_downscale_factor=2,
        min_mask_region_area=50      # Bajar el filtro de "basura" a nivel de parche
    )
    print("Generador automático de máscaras configurado.")
else:
    print("No se pudo cargar el modelo. El archivo checkpoint no se encontró.")

Cargando modelo SAM en el dispositivo: cuda
Modelo cargado exitosamente.
Configurando generador con parámetros más relajados...
Generador automático de máscaras configurado.


In [21]:
# --- 4.1. Definir Parámetros de Parcheo ---
# 1024 es el tamaño de entrada nativo del codificador de SAM
PATCH_SIZE = 3000
# 200px de superposición es robusto para la fusión.
OVERLAP = 200
# Tu límite de área mínima (en píxeles).
# Polígonos más pequeños que esto serán descartados.
MIN_AREA_THRESHOLD = 5000

# Tomar la imagen ajustada del Bloque 2 y convertirla a numpy RGB
# El generador de SAM espera un formato 'uint8' [H, W, 3]
try:
    img_to_segment = np.array(adjusted_image_pil.convert('RGB'))
    img_h, img_w, _ = img_to_segment.shape
    print(f"Dimensiones de la imagen a segmentar: {img_to_segment.shape}")

    all_polygons = [] # Lista para guardar todos los polígonos (de Shapely)

    print(f"Iniciando segmentación por parches (Tamaño: {PATCH_SIZE}, Superposición: {OVERLAP})...")

    # --- 4.2. Iterar sobre los Parches ---
    for y in range(0, img_h, PATCH_SIZE - OVERLAP):
        for x in range(0, img_w, PATCH_SIZE - OVERLAP):
            # Definir los límites del parche
            y1, y2 = y, min(y + PATCH_SIZE, img_h)
            x1, x2 = x, min(x + PATCH_SIZE, img_w)

            # Recortar el parche
            patch = img_to_segment[y1:y2, x1:x2]

            # Omitir parches que sean más pequeños que la superposición
            if patch.shape[0] < OVERLAP or patch.shape[1] < OVERLAP:
                continue

            print(f"Procesando parche en (x, y): ({x1}, {y1}) con tamaño {patch.shape}")

            # --- 4.3. Ejecutar SAM en el Parche ---
            # ¡Este es el paso que consume tiempo!
            masks = mask_generator.generate(patch)

            if not masks:
                continue

            # --- 4.4. Filtrar y Convertir Máscaras del Parche ---
            for mask_data in masks:
                area = mask_data['area']

                # 1. Filtrar por tu área mínima
                if area < MIN_AREA_THRESHOLD:
                    continue

                # 2. Convertir máscara (booleano) a contornos (polígonos)
                segmentation = mask_data['segmentation']
                contours, _ = cv2.findContours(segmentation.astype(np.uint8),
                                               cv2.RETR_EXTERNAL,
                                               cv2.CHAIN_APPROX_SIMPLE)

                for contour in contours:
                    if contour.shape[0] < 3: # Un polígono necesita al menos 3 puntos
                        continue

                    # 3. Re-proyectar contorno a coordenadas globales
                    # Se suma el offset (x1, y1) del parche
                    global_contour = contour.squeeze() + [x1, y1]

                    # 4. Crear polígono de Shapely
                    polygon = Polygon(global_contour)

                    # 5. Corregir geometrías inválidas si es necesario
                    if not polygon.is_valid:
                        polygon = polygon.buffer(0)

                    if polygon.is_valid and not polygon.is_empty:
                        all_polygons.append(polygon)

    print(f"Segmentación completada. {len(all_polygons)} polígonos crudos encontrados.")

    # --- 4.5. Fusionar (Unir) todos los Polígonos ---
    print("Iniciando fusión de polígonos superpuestos...")
    if all_polygons:
        # unary_union disuelve los bordes internos de todos los polígonos
        # que se superponen (de los parches adyacentes).
        merged_polygons = unary_union(all_polygons)
        print("Fusión completada.")

        # El resultado puede ser un solo Polígono o un MultiPolígono
        if merged_polygons.geom_type == 'Polygon':
            final_polygons = [merged_polygons]
        else:
            # Convertir el MultiPolygon en una lista de Polígonos
            final_polygons = list(merged_polygons.geoms)

        print(f"Total de polígonos finales fusionados: {len(final_polygons)}")
    else:
        final_polygons = []
        print("No se encontraron polígonos que cumplan los criterios.")

except Exception as e:
    print(f"Ocurrió un error durante la segmentación: {e}")
    final_polygons = [] # Asegurarse de que la variable exista para el Bloque 5

Dimensiones de la imagen a segmentar: (3025, 3032, 3)
Iniciando segmentación por parches (Tamaño: 3000, Superposición: 200)...
Procesando parche en (x, y): (0, 0) con tamaño (3000, 3000, 3)
Ocurrió un error durante la segmentación: CUDA out of memory. Tried to allocate 4.70 GiB. GPU 0 has a total capacity of 14.74 GiB of which 4.39 GiB is free. Process 10613 has 10.35 GiB memory in use. Of the allocated memory 8.96 GiB is allocated by PyTorch, and 1.26 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)


In [22]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm # Importamos la biblioteca de Colormaps
from matplotlib.patches import Polygon as MplPolygon
from matplotlib.collections import PatchCollection

print("Mostrando resultados finales (Polígonos con Colormap vs. Original)...")

# --- 5.1. Crear la Figura con 2 Sub-gráficos ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))

# --- 5.2. Gráfico 1: Polígonos con Relleno de Colormap ---

# Establecemos un fondo oscuro (negro)
ax1.set_facecolor('black')

patches_to_draw = []
if final_polygons:
    num_polygons = len(final_polygons)

    # --- ¡CAMBIO PRINCIPAL AQUÍ! ---
    # Obtenemos un Colormap.
    # 'nipy_spectral' o 'gist_rainbow' ofrecen una gran variedad de colores.
    # Obtenemos el colormap y le decimos cuántos colores necesitamos.
    colormap = cm.get_cmap('nipy_spectral', num_polygons)

    for i, poly in enumerate(final_polygons):
        # Obtener las coordenadas exteriores del polígono de Shapely
        x, y = poly.exterior.xy

        # Asignar un color basado en el índice del polígono (i).
        # Normalizamos 'i' dividiéndolo por el total de polígonos.
        color_del_mapa = colormap(i / num_polygons)

        # Crear un polígono de Matplotlib con relleno
        mpl_poly = MplPolygon(
            np.column_stack((x, y)),
            fill=True,
            facecolor=color_del_mapa, # Aplicar el color del colormap
            edgecolor='white',       # Borde blanco fino para separar
            linewidth=0.5,           # Ancho de línea muy fino
            alpha=0.85               # Un poco más opaco
        )
        patches_to_draw.append(mpl_poly)

    # Añadir la colección de polígonos al gráfico 1
    p = PatchCollection(patches_to_draw, match_original=True)
    ax1.add_collection(p)

# --- FIJAR LÍMITES (Igual que antes) ---
# Establecer los límites del gráfico para que coincidan con la imagen original
ax1.set_ylim(0, img_h)
ax1.set_xlim(0, img_w)
ax1.invert_yaxis() # Invertir el eje Y (0 arriba)

ax1.set_title(f"Resultados: {len(final_polygons)} Polígonos (Colormap)")
ax1.axis('off') # Ocultar los ejes

# --- 5.3. Gráfico 2: Imagen Original ---

# Mostrar la imagen original (cargada en el Bloque 1, variable 'img_rgb')
ax2.imshow(img_rgb)
ax2.set_title("Imagen Original")
ax2.axis('off')

# --- 5.4. Mostrar el Gráfico ---
plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.